# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 1 2022 3:59PM,240983,19,ADV80003754,"AdvaGen Pharma, Ltd",Released
1,Jul 1 2022 3:59PM,240983,19,ADV80003755,"AdvaGen Pharma, Ltd",Released
2,Jul 1 2022 3:59PM,240983,19,ADV80003756,"AdvaGen Pharma, Ltd",Released
3,Jul 1 2022 3:59PM,240983,19,ADV80003757,"AdvaGen Pharma, Ltd",Released
4,Jul 1 2022 3:59PM,240983,19,ADV80003758,"AdvaGen Pharma, Ltd",Released
5,Jul 1 2022 3:59PM,240983,19,ADV80003759,"AdvaGen Pharma, Ltd",Released
6,Jul 1 2022 3:59PM,240983,19,ADV80003760,"AdvaGen Pharma, Ltd",Released
7,Jul 1 2022 3:59PM,240983,19,ADV80003761,"AdvaGen Pharma, Ltd",Released
8,Jul 1 2022 3:59PM,240983,19,ADV80003762,"AdvaGen Pharma, Ltd",Released
9,Jul 1 2022 3:59PM,240983,19,ADV80003763,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
3,240956,Executing,18
4,240956,Released,7
5,240959,Executing,11
6,240978,Released,5
7,240983,Released,18


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240949,1.0,NaN
240956,18.0,7.0
240959,11.0,NaN
240978,NaN,5.0
240983,NaN,18.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240918,0.0,6.0
240932,0.0,1.0
240949,1.0,0.0
240956,18.0,7.0
240959,11.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240918,0,6
240932,0,1
240949,1,0
240956,18,7
240959,11,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240918,0,6
1,240932,0,1
2,240949,1,0
3,240956,18,7
4,240959,11,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240918,,6
1,240932,,1
2,240949,1,
3,240956,18,7
4,240959,11,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 1 2022 3:59PM,240983,19,"AdvaGen Pharma, Ltd"
18,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc."
23,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc."
34,Jul 1 2022 12:06PM,240956,20,Alumier Labs Inc.
59,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc."
60,Jul 1 2022 9:09AM,240932,10,"SD Head USA, LLC"
61,Jul 1 2022 8:14AM,240918,16,Sartorius Bioprocess Solutions
64,Jul 1 2022 8:14AM,240918,16,Sartorius Lab Products and Service


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 1 2022 3:59PM,240983,19,"AdvaGen Pharma, Ltd",,18
1,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",,5
2,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",11,
3,Jul 1 2022 12:06PM,240956,20,Alumier Labs Inc.,18,7
4,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc.",1,
5,Jul 1 2022 9:09AM,240932,10,"SD Head USA, LLC",,1
6,Jul 1 2022 8:14AM,240918,16,Sartorius Bioprocess Solutions,,6
7,Jul 1 2022 8:14AM,240918,16,Sartorius Lab Products and Service,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 3:59PM,240983,19,"AdvaGen Pharma, Ltd",18,
1,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",5,
2,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",,11
3,Jul 1 2022 12:06PM,240956,20,Alumier Labs Inc.,7,18
4,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc.",,1
5,Jul 1 2022 9:09AM,240932,10,"SD Head USA, LLC",1,
6,Jul 1 2022 8:14AM,240918,16,Sartorius Bioprocess Solutions,6,
7,Jul 1 2022 8:14AM,240918,16,Sartorius Lab Products and Service,6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 3:59PM,240983,19,"AdvaGen Pharma, Ltd",18,
1,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",5,
2,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",,11
3,Jul 1 2022 12:06PM,240956,20,Alumier Labs Inc.,7,18
4,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc.",,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 3:59PM,240983,19,"AdvaGen Pharma, Ltd",18.0,NaN
1,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",5.0,NaN
2,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",NaN,11.0
3,Jul 1 2022 12:06PM,240956,20,Alumier Labs Inc.,7.0,18.0
4,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc.",NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 3:59PM,240983,19,"AdvaGen Pharma, Ltd",18.0,0.0
1,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",5.0,0.0
2,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",0.0,11.0
3,Jul 1 2022 12:06PM,240956,20,Alumier Labs Inc.,7.0,18.0
4,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc.",0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,481910,6.0,0.0
16,481836,12.0,0.0
19,240983,18.0,0.0
20,481915,7.0,29.0
21,240949,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,481910,6.0,0.0
1,16,481836,12.0,0.0
2,19,240983,18.0,0.0
3,20,481915,7.0,29.0
4,21,240949,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,6.0,0.0
1,16,12.0,0.0
2,19,18.0,0.0
3,20,7.0,29.0
4,21,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,6.0
1,16,Released,12.0
2,19,Released,18.0
3,20,Released,7.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Executing,0.0,0.0,0.0,29.0,1.0
Released,6.0,12.0,18.0,7.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Executing,0.0,0.0,0.0,29.0,1.0
1,Released,6.0,12.0,18.0,7.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Executing,0.0,0.0,0.0,29.0,1.0
1,Released,6.0,12.0,18.0,7.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()